In [6]:
# 두번째 수행평가!!
# 코드를 작성해서 kaggle에 올리기까지만
# Titanic Data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats  # 이상치 처리를 위해 필요
import tensorflow as tf  # tensorflow 구현
from sklearn import linear_model   # sklearn으로 simple linear regression 구현
from sklearn.preprocessing import MinMaxScaler


df_train = pd.read_csv('./data/titanic/train.csv')  # PassengerId 1 ~ 892
df_test = pd.read_csv('./data/titanic/test.csv') # PassengerId 892~

df = pd.concat([df_train, df_test], axis=0, ignore_index='True')
# display(df) # 1309 rows × 12 columns



# 데이터 전처리

# PassengerId : 이 아이디가 생존여부가 연관성이 있나? 
#               숫자 자체로는 종속변수에 영향을 미치지 않음
#               머신러닝에 넣을 필요가 없음.


# Survived : 종속변수(label) 0과 1로 되어 있음. 그대로 사용


# Pclass : 숫자로 되어있음. 영향을 미침


# Name : 이름자체는 생존여부에 영향을 미치지 않음. 그러나 나이 결측지 판별에 도움
df['Initial'] = 0
for i in df:
    df['Initial']=df.Name.str.extract('([A-Za-z]+)\.')
# display(df['Initial'])
# display(pd.crosstab(df['Initial'], df['Sex']))
df['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don','Dona'],
                      ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr','Master'],
                      inplace=True)
# display(df['Initial'])


# Sex : 생존 영향에 끼침. male, female 문자로 되어 있으니 숫자로 변환.


# Age : 나이에 결측치가 있음. 삭제보다는 값을 대체시켜서 사용하는게 좋아보임.
#       이름 부분에 호칭을 이용해서 결혼유무를 보고 나이를 예측. ex) 미혼여성에대한 평균
#       0~15 : 소아(0), 16~24 : 청년(1), 25 ~45 : 성인(2), 46~64 : 중년(3), 65~ : 노인(4)
# display(df.groupby('Initial')['Age'].mean())
df.loc[(df.Age.isnull())&(df.Initial=='Master'), 'Age'] =5
df.loc[(df.Age.isnull())&(df.Initial=='Miss'), 'Age'] =22
df.loc[(df.Age.isnull())&(df.Initial=='Mr'), 'Age'] =33
df.loc[(df.Age.isnull())&(df.Initial=='Mrs'), 'Age'] =36
df.loc[(df.Age.isnull())&(df.Initial=='Other'), 'Age'] =46
# print('최고령자 나이',df['Age'].values.max())  # 80세

# Age의 경우에는 연속된 수로 되어 있기 때문에 Machine Learning 모델에서는 적합하지 않다.
# 그래서 Age를 구간으로 나누어서 Machine Learning에서 판단할 수 있도록 정해야 한다.
df['Age_group'] = 0
df.loc[df['Age']<16, 'Age_group']=0
df.loc[(df['Age']>=16)&(df['Age']<25), 'Age_group']=1
df.loc[(df['Age']>=25)&(df['Age']<46), 'Age_group']=2
df.loc[(df['Age']>=46)&(df['Age']<65), 'Age_group']=3
df.loc[(df['Age']>=65), 'Age_group']=4

# display(df['Age_group'].value_counts())
# 2    688
# 1    345
# 3    140
# 0    123
# 4     13



# parch and sibsp
df['Family']=0
df['Family']=df['Parch']+df['SibSp']
df['Alone']=0
df.loc[df.Family==0,'Alone']=1


# ticket : 영향이 있으나 다른 변수들과 중복되어 보임 / 제외 결정


# fare : class, 거리 등등 요소들이 묶여있을거임
# display(df['Fare'])
df['Fare_Range']=pd.qcut(df['Fare'],4)
# display(df.groupby(['Fare_Range'])['Survived'].mean())
df['Fare_cut']=0
df.loc[df['Fare']<=7.91,'Fare_cut']=0
df.loc[(df['Fare']>7.91)&(df['Fare']<=14.454),'Fare_cut']=1
df.loc[(df['Fare']>14.454)&(df['Fare']<=31),'Fare_cut']=2
df.loc[(df['Fare']>31)&(df['Fare']<=513),'Fare_cut']=3


# cabin : 객실번호, (제외, Nan을 수정해서 사용할지는 결정.) / 제외 결정


# Embarked : 어디서 탔느냐, 데이터분석을 해보면 (상관분석) 영향이 있음  => 
# 많은 승객들이 S에서 승선한것을 알 수 있기 때문에 
# Embarked에 Null인 데이터는 S에서 승선한것으로 채워넣는다


# 제외할 데이터 처리 
# passengerid, ticket, cabin
passengerId = df['PassengerId'] # 제출을 위해 빼놓기
df.drop(['Name','Age','Fare','Fare_Range','Ticket','PassengerId','Cabin'], axis=1, inplace=True)
# display(df) # 1309 rows × 11 columns / PassengerId 제외, 나머지 대체


# 결측치 처리
# print(df.isnull().sum()) # age, Embarked에 결측치가 존재
df['Embarked'].fillna('S',inplace=True) # S로 채움
# print(df.isnull().sum()) # 결측치 처리 완료.


# 데이터 수치화 
# Sex, Embarked, Initial 수치화
# display(df)
df['Sex'].replace(['male','female'], [0,1], inplace=True)
# mapping = {'male : 0, 'female' : 1}
# df['Sex] = df['Sex'].map(mapping) 로 대체 가능


df['Embarked'].replace(['S','Q','C'], [0,1,2], inplace=True)
df['Initial'].replace(['Mr','Mrs','Miss','Master','Other'], [0,1,2,3,4], inplace=True)
display(df) # 수치화 완료


# 정규화 
# 굳이 정규화 안해도 될듯
# x_data = df.drop('Survived', axis=1, inplace=False).values
# t_data = df['Survived'].values.reshape(-1,1)

# scaler_x = MinMaxScaler()  # 객체 생성
# scaler_x.fit(x_data)
# training_data_x = scaler_x.transform(x_data)

# display(df.loc[df['PassengerId'] <= 891])
# display(df.loc[df['PassengerId'] == 892]) # raw 891 부터

# train & test split
tr_data = df[:891]
test_data = df[891:]
display(test_data)

,Survived,Pclass,Sex,SibSp,Parch,Embarked,Initial,Age_group,Family,Alone,Fare_cut
0,0.0,3,0,1,0,0,0,1,1,0,0
1,1.0,1,1,1,0,2,1,2,1,0,3
2,1.0,3,1,0,0,0,2,2,0,1,1
3,1.0,1,1,1,0,0,1,2,1,0,3
4,0.0,3,0,0,0,0,0,2,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
1304,NaN,3,0,0,0,0,0,2,0,1,1
1305,NaN,1,1,0,0,2,3,2,0,1,3
1306,NaN,3,0,0,0,0,0,2,0,1,0
1307,NaN,3,0,0,0,0,0,2,0,1,1


,Survived,Pclass,Sex,SibSp,Parch,Embarked,Initial,Age_group,Family,Alone,Fare_cut
891,NaN,3,0,0,0,1,0,2,0,1,0
892,NaN,3,1,1,0,0,1,3,1,0,0
893,NaN,2,0,0,0,1,0,3,0,1,1
894,NaN,3,0,0,0,0,0,2,0,1,1
895,NaN,3,1,1,1,0,1,1,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1304,NaN,3,0,0,0,0,0,2,0,1,1
1305,NaN,1,1,0,0,2,3,2,0,1,3
1306,NaN,3,0,0,0,0,0,2,0,1,0
1307,NaN,3,0,0,0,0,0,2,0,1,1


In [7]:
# 학습과 validation을 수행해야 됨.
# 데이터를 7:3 비율로 Training Data Set과 Validation Data Set으로 분리
train_data = tr_data.iloc[:int(tr_data.shape[0] * 0.7)]
val_data = tr_data.iloc[int(tr_data.shape[0] * 0.7):]

# Training Data Set
train_x_data = train_data.drop('Survived', axis=1, inplace=False).values.reshape(-1,10)
train_t_data = train_data['Survived'].values.reshape(-1,1)

# Validation Data Set
val_x_data = val_data.drop('Survived', axis=1, inplace=False).values.reshape(-1,10)
val_t_data = val_data['Survived'].values.reshape(-1,1)

# Test Data Set
test_x_data = test_data.drop('Survived', axis=1, inplace=False).values.reshape(-1,10)

In [22]:
# tensorflow 구현

# Placeholder
X = tf.placeholder(shape=[None, 10], dtype=tf.float32)
T = tf.placeholder(shape=[None, 1], dtype=tf.float32)

# Weight & bias
W = tf.Variable(tf.random.normal([10,1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

# Hypothesis
logit = tf.matmul(X,W) + b
H = tf.sigmoid(logit)

# loss function
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=T))

# train # GradientDescent 알고리즘을 Optimizer(최적화) 시킴
train = tf.train.GradientDescentOptimizer(learning_rate=1e-4).minimize(loss)

# Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
for step in range(100000):
    _, W_val, b_val, loss_val = sess.run([train, W, b, loss], 
                                         feed_dict={X: train_x_data, 
                                                    T : train_t_data})
    
    if step % 10000 == 0:
        print('W : {}, b : {}, loss : {}'.format(W_val.ravel(),b_val,loss_val))

W : [-1.2046362  -1.0341713  -0.7265818   0.21141993 -0.01171854 -0.52997255
  0.6195971   0.78368187 -0.40369144  0.19444606], b : [-0.20631136], loss : 1.2166997194290161
W : [-0.91444945 -0.84577936 -0.71928984  0.20574687  0.07563425 -0.18279299
  0.6656593   0.78530115 -0.33767867  0.3081695 ], b : [-0.09654317], loss : 0.8970562219619751
W : [-0.8071643  -0.6998938  -0.76341337  0.15861341  0.11541308  0.0696037
  0.57779366  0.6940314  -0.31379065  0.2697355 ], b : [-0.07072623], loss : 0.775053858757019
W : [-0.7364157  -0.5709955  -0.79693043  0.11041163  0.14322752  0.27974272
  0.48177788  0.6123172  -0.30142274  0.22206949], b : [-0.05825467], loss : 0.6863437294960022
W : [-0.68386704 -0.45632777 -0.81821054  0.06555861  0.16439399  0.45418882
  0.3957675   0.5461827  -0.29514992  0.18281326], b : [-0.049812], loss : 0.6234787106513977
W : [-0.6460298  -0.35484305 -0.8320114   0.02486423  0.18065503  0.5965689
  0.32283986  0.49168688 -0.2931939   0.15417245], b : [-0.0435

In [23]:
# 정확도(Accuracy) 측정
# 실행하면 확률을 내보내줌 True, False로 나오는데 이를 0,1로 변경
# cast함수를 이용해 dtype을 bool -> 실수로 바꿔줌
predict = tf.cast(H > 0.5, dtype=tf.float32)  # 예측값 : [ 1 0 0 0 1 0 1 0]

# 내가 예측한것과 T 데이터를 가지고 비교
correct = tf.equal(predict, T)       # 만약 t값이 : [ 1 1 0 0 1 ] => 대입하면 맞춘건 True 틀린건 False[True, False]
accuray = tf.reduce_mean(tf.cast(correct, dtype=tf.float32)) # 그럼 맞춘건 1 틀린건 0이기 때문에 평균을 내면 정확도가 된다!

accuray_val = sess.run(accuray, feed_dict={X : val_x_data,
                                           T : val_t_data}) # val_t_data
print('모델의 정확도 : {}'.format(accuray_val))


모델의 정확도 : 0.7947761416435242


In [9]:
# 제출 하기

submission = pd.read_csv('./data/titanic/gender_submission.csv')

submission['Survived'] = sess.run(predict, feed_dict={X:test_x_data}).astype(np.int64)
submission.to_csv('./data/titanic/submission2.csv', index=False, encoding='utf-8')

